In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import json
import logging
import math
import os

from matplotlib import pyplot as plt
import pandas as pd
import papermage
from papermage.recipes import CoreRecipe
from papermage.visualizers import plot_entities_on_page
from tqdm.auto import tqdm

from papermage_components.materials_recipe import MaterialsRecipe

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
recipe = MaterialsRecipe()

INFO:MaterialsRecipe:Instantiating recipe...
INFO:iopath.common.file_io:URL https://www.dropbox.com/s/ukbw5s673633hsw/publaynet-tf_efficientdet_d0.pth.tar?dl=1 cached in /Users/sireeshgururaja/.torch/iopath_cache/s/ukbw5s673633hsw/publaynet-tf_efficientdet_d0.pth.tar?dl=1
INFO:timm.models._helpers:Loaded state_dict from checkpoint '/Users/sireeshgururaja/.torch/iopath_cache/s/ukbw5s673633hsw/publaynet-tf_efficientdet_d0.pth.tar?dl=1'
/Users/sireeshgururaja/miniconda3/envs/papermage/lib/python3.11/site-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
INFO:MaterialsRecipe:Finished instantiating recipe


In [7]:
paper_list = [
    recipe.from_pdf(f"data/AM_Creep_Papers/{paper_filename}")
    for paper_filename in tqdm(os.listdir("data/AM_Creep_Papers/")[:1])
]

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:MaterialsRecipe:Parsing document...
INFO:MaterialsRecipe:Rasterizing document...
INFO:MaterialsRecipe:Predicting words...
INFO:MaterialsRecipe:Predicting sentences...


AttributeError: 'SciSpacySentencePredictor' object has no attribute 'spacy'

In [ ]:
print("\n\n".join([paper.titles[0].text.replace("\n", " ") for paper in paper_list]))

In [ ]:
def highlight_relevant_entities(document):
    page_images = []
    for page in document.pages:
        highlighted = page.images[0]
        # highlighted = plot_entities_on_page(highlighted, page.tokens, box_width=0, box_alpha=0.3, box_color="yellow")
        highlighted = plot_entities_on_page(
            highlighted, page.abstracts, box_width=2, box_alpha=0.1, box_color="orange"
        )
        highlighted = plot_entities_on_page(
            highlighted, page.captions, box_width=2, box_alpha=0.1, box_color="orange"
        )

        highlighted = plot_entities_on_page(
            highlighted, page.tables, box_width=2, box_alpha=0.1, box_color="yellow"
        )
        highlighted = plot_entities_on_page(
            highlighted, page.figures, box_width=2, box_alpha=0.1, box_color="yellow"
        )

        highlighted = plot_entities_on_page(
            highlighted, page.sections, box_width=2, box_alpha=0.1, box_color="green"
        )
        highlighted = plot_entities_on_page(
            highlighted, page.titles, box_width=2, box_alpha=0.1, box_color="green"
        )

        highlighted = plot_entities_on_page(
            highlighted, page.blocks, box_width=2, box_alpha=0.1, box_color="grey"
        )

        page_images.append(highlighted)
    return page_images

In [ ]:
paper_iter = iter(paper_list)

In [ ]:
sample_paper.sections[2].sentences

In [ ]:
sample_paper.layers

In [ ]:
sample_paper.sections[2]

In [ ]:
sample_paper = next(paper_iter)
print(sample_paper.titles[0].text)

highlighted_doc = highlight_relevant_entities(sample_paper)

n_cols = 3
n_rows = math.ceil(len(highlighted_doc) / 3)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(17, 22), tight_layout=True)

for ax in itertools.chain(*axes):
    ax.set_axis_off()

for ax, image in zip(itertools.chain(*axes), highlighted_doc):
    ax.imshow(image.to_array())

In [ ]:
for sentence in sample_paper.sentences:
    print(sentence.text.replace("\n", " "))
    print("\n")

In [ ]:
for paper in paper_list:
    paper_title = ""
    for title in paper.titles:
        if len(title.text) > len(paper_title):
            paper_title = title.text
    paper_sentences = [sentence.text.replace("\n", " ") for sentence in paper.sentences]
    paper_abstract = paper.abstracts[0].text
    section_text = {}

    with open(f"data/AM_Creep_Papers_parsed/{paper_title.replace(' ', '_')}.json", "w") as f:
        json.dump(
            {
                "title": paper_title,
                "abstract": paper_abstract,
                "sentences": paper_sentences,
                "section_text": section_text,
            },
            f,
            indent=4,
        )

In [ ]:
type(paper_list[0])

In [ ]:
from papermage import Document

## NOTES:

- Titles are _sometimes_ (but rarely) flaky
- Figures are borked; we're really not getting much utility.
- Captions are a little better than figures, but there's a weird level of caption agglomeration going on. Not great, but still probably workable for tables.
- Tables seem, to be working fine!!
- a section is occasionally missed